In [ ]:
import glob
import numpy
import pandas
import seaborn
import matplotlib.pyplot as plt
from tqdm import tqdm
import multiprocessing as mp
import os
from build import build_model
import coralme

In [ ]:
from IPython.display import display, HTML, Math, Markdown
display(HTML("<style>.container { width:95% !important; }</style>"))

%load_ext autoreload
%autoreload 2

In [ ]:
organisms = {i.split(".gb")[0] for i in os.listdir("./agora-models/genbank-mixed-ncbi-agora-sources/")}
len(organisms)

In [ ]:
!bash report.sh
step1 = set(pandas.read_csv("step1.txt",index_col=0,header=None).index.to_list())
step2 = set(pandas.read_csv("step2.txt",index_col=0,header=None).index.to_list())
try:
    step3 = set(pandas.read_csv("step3.txt",index_col=0,header=None).index.to_list())
except:
    step3 = set()
no_overlap = set(pandas.read_csv("no_overlap.txt",index_col=0,header=None).index.to_list())
trna_issues = set(pandas.read_csv("trna_issues.txt",index_col=0,header=None).index.to_list())
trna_ts_issues = set(pandas.read_csv("trna_ts_issues.txt",index_col=0,header=None).index.to_list())

In [ ]:
model = models["Listeria_monocytogenes_SLCC_2540"]

In [ ]:
def get_partition(m):
    df = flux_based_reactions(m._model,m.id)["met_flux"]
    df = df[df<0]
    df = df/df.sum()
    dct = df.to_dict()
    return {model.get(r):flux for r,flux in dct.items()}

In [ ]:
from allocation import get_allocation,get_subsystem,find_complexes

In [ ]:
dct = {}
for m in tqdm(model.metabolites.query("^RNA_")):
    dct[m.id] = {i.id:j for i,j in find_complexes(m, seen = dict(),multiplier=1).items()}
df = pandas.DataFrame.from_dict(dct).T

In [ ]:
df.fillna(0)